In [ ]:
from functions import *

# greeks("NVDA", '2026-01-16', 175, "put")

In [ ]:
continuous_monitor("AMC")

In [ ]:
greeks("AMC", '2026-01-16', 2, "call")

In [ ]:
option_price("AMC", '2026-01-16', 2, "call")

In [ ]:
# Get historical stock price
price = get_stock_price_historical('AAPL', '2024-01-15')
print(f"AAPL price on 2024-01-15: ${price}")

# Calculate theoretical option price on a historical date
option_price = option_price_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15'
)
print(f"Theoretical call price: ${option_price}")

# Calculate Greeks on a historical date
greeks_data = greeks_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15'
)
print(f"Historical Greeks: {greeks_data}")

# With custom implied volatility
option_price = option_price_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15', iv=0.25
)
print(f"Price with 25% IV: ${option_price}")


In [ ]:
# Get intraday stock price
price = get_stock_price_intraday('AAPL', '2025-12-01', '14:30')
print(f"AAPL at 2:30 PM: ${price}")

# Calculate option price at a specific time
option_price = option_price_intraday(
    'AAPL', '2026-01-16', 150.0, 'call', 
    '2025-12-01', '14:30'
)
print(f"Call option price at 2:30 PM: ${option_price}")

# Calculate Greeks at a specific time
greeks = greeks_intraday(
    'AAPL', '2026-01-16', 150.0, 'call',
    '2025-12-01', '14:30'
)
print(f"Greeks at 2:30 PM: \n{greeks}")

# With custom IV
greeks = greeks_intraday(
    'AAPL', '2026-01-16', 150.0, 'call',
    '2025-12-01', '14:30', iv=0.28
)
print(f"Greeks at 2:30 PM with IV = 28%: \n{greeks}")

In [ ]:
from functions import *

# Simulate buying a call option with 20% stop-loss and 50% take-profit
result = options_purchase(
    'AAPL', 
    '2026-01-16',  # Expiration
    150.0,          # Strike
    '2025-12-10',   # Purchase date
    '10:30',        # Purchase time (10:30 AM ET)
    'call',
    stoploss=20,    # -20% stop-loss
    takeprofit=50   # +50% take-profit
)

print(f"Entry: ${result['entry_price']}")
print(f"Exit: ${result['exit_price']} at {result['exit_time']}")
print(f"Reason: {result['exit_reason']}")
print(f"P/L: {result['pnl_percent']}% (${result['pnl_dollar']} per contract)")


In [ ]:
from functions import *

# Simulate buying a call option with 20% stop-loss and 50% take-profit
result = options_purchase(
    'AMC', 
    '2026-01-16',  # Expiration
    2,          # Strike
    '2025-12-16',   # Purchase date
    '09:30',        # Purchase time (09:30 AM ET)
    'call',
    stoploss=20,    # -20% stop-loss
    takeprofit=50   # +50% take-profit
)

print(f"Entry: ${result['entry_price']}")
print(f"Exit: ${result['exit_price']} at {result['exit_time']}")
print(f"Reason: {result['exit_reason']}")
print(f"P/L: {result['pnl_percent']}% (${result['pnl_dollar']} per contract)")


In [ ]:
from functions import *

In [2]:
df = pd.read_csv('nasdaq_tickers.csv')
#df = df.to_dict('records')

In [3]:
#df[['Ticker']]

In [13]:
from functions import *

reasonable_stocks = []

for n in range(0,600):
    ticker, price = list(df[['Ticker','Last Price']].iloc[n])
    if 3*price < 100:
        reasonable_stocks.append([ticker, price])

reasonable_stocks = [[ticker, float(price)] for ticker, price in reasonable_stocks]

In [28]:
reasonable_stocks[0]

def check_option(ticker, price):
    try:
        # Get the option chain to see available strikes
        ticker_obj = yf.Ticker(ticker)
        chain = ticker_obj.option_chain('2026-01-16')
        
        # Check if option chain has data
        if chain.calls.empty or chain.puts.empty:
            print(f"{ticker} - No option data available")
            return None
        
        # Find nearest strike for call (above current price)
        call_strikes = chain.calls['strike'].values
        call_target = price * 1.01
        call_strike = call_strikes[np.argmin(np.abs(call_strikes - call_target))]
        
        # Find nearest strike for put (below current price)
        put_strikes = chain.puts['strike'].values
        put_target = price * 0.99
        put_strike = put_strikes[np.argmin(np.abs(put_strikes - put_target))]
        
        call_value = option_price(ticker, '2026-01-16', call_strike, 'call')
        put_value = option_price(ticker, '2026-01-16', put_strike, 'put')
        print(f"{ticker} (${price:.2f})\n  Call ${call_strike}: {call_value}\n  Put ${put_strike}: {put_value}")
        return {'call': call_value, 'put': put_value}
    except Exception as e:
        print(f"{ticker} - Error: {type(e).__name__}: {e}")
        return None

In [31]:
for ticker, price in reasonable_stocks[0:10]:
    check_option(ticker, price)
    print("")

CMCSA ($29.73)
  Call $30.0: {'lastPrice': 0.9, 'bid': 0.87, 'ask': 0.91, 'volume': 1542}
  Put $30.0: {'lastPrice': 1.23, 'bid': 1.24, 'ask': 1.3, 'volume': 222}

NU ($16.20)
  Call $16.0: {'lastPrice': 0.48, 'bid': 0.49, 'ask': 0.51, 'volume': 837}
  Put $16.0: {'lastPrice': 0.69, 'bid': 0.67, 'ask': 0.69, 'volume': 236}

KDP ($28.59)
  Call $29.0: {'lastPrice': 0.43, 'bid': 0.4, 'ask': 0.5, 'volume': 25}
  Put $28.0: {'lastPrice': 0.65, 'bid': 0.6, 'ask': 0.65, 'volume': 6}

SOFI ($26.58)
  Call $27.0: {'lastPrice': 1.22, 'bid': 1.2, 'ask': 1.22, 'volume': 1511}
  Put $26.0: {'lastPrice': 1.65, 'bid': 1.66, 'ask': 1.69, 'volume': 776}

KHC ($24.68)
  Call $25.0: {'lastPrice': 0.63, 'bid': 0.6, 'ask': 0.65, 'volume': 136}
  Put $25.0: {'lastPrice': 0.62, 'bid': 0.6, 'ask': 0.64, 'volume': 26}

PSNY - Error: ValueError: cannot convert float NaN to integer

HPQ ($24.83)
  Call $25.0: {'lastPrice': 0.57, 'bid': 0.55, 'ask': 0.58, 'volume': 32}
  Put $25.0: {'lastPrice': 1.15, 'bid': 1.1

In [35]:
greeks("AMC", '2026-01-16', 2, "call", status = True);


GREEKS ANALYSIS: AMC CALL $2.0 exp 2026-01-16

CURRENT MARKET DATA
   Stock Price: $1.89
   Strike Price: $2.00
   Days to Expiration: 29
   Implied Volatility: 82.81%
   Moneyness: OTM (Out-of-The-Money) by $0.11 (5.25%)

RAW GREEKS VALUES
   Delta (Δ):     0.4593
   Gamma (Γ):     0.8972
   Vega (ν):      0.0021
   Theta (Θ):    -0.0031
   Rho (ρ):       0.0006

DELTA ANALYSIS
   Probability of Expiring ITM: ~45.9%
   Equivalent Share Position: 45.93 shares
   Delta Classification: ATM - High directional exposure
   Direction: Bullish

GAMMA ANALYSIS
   Delta Change per $1 Move: 0.8972
   Gamma Risk Level: Very High - Extreme delta sensitivity
   Distance from Max Gamma (ATM): 5.54%

VEGA ANALYSIS
   P&L per 1% IV Change: $0.00
   Volatility Exposure: Long Volatility
   Vega Classification: Low IV Sensitivity

THETA ANALYSIS
   Daily Time Decay: $-0.00
   Weekly Time Decay: $-0.02
   Monthly Time Decay (30d): $-0.09
   Theta Classification: Low - Minimal time decay
   Stock Move to 

In [13]:
option_price("INTC", '2026-01-02', 37, "call")

{'lastPrice': 1.08, 'bid': 1.1, 'ask': 1.18, 'volume': 249}

In [15]:
greeks("INTC", '2026-01-02', 37, "call", status=True)


GREEKS ANALYSIS: INTC CALL $37.0 exp 2026-01-02

CURRENT MARKET DATA
   Stock Price: $36.46
   Strike Price: $37.00
   Days to Expiration: 15
   Implied Volatility: 45.41%
   Moneyness: ATM (At-The-Money) within $0.54

RAW GREEKS VALUES
   Delta (Δ):     0.4604
   Gamma (Γ):     0.1183
   Vega (ν):      0.0293
   Theta (Θ):    -0.0459
   Rho (ρ):       0.0064

DELTA ANALYSIS
   Probability of Expiring ITM: ~46.0%
   Equivalent Share Position: 46.04 shares
   Delta Classification: ATM - High directional exposure
   Direction: Bullish

GAMMA ANALYSIS
   Delta Change per $1 Move: 0.1183
   Gamma Risk Level: Very High - Extreme delta sensitivity
   Distance from Max Gamma (ATM): 1.49%

VEGA ANALYSIS
   P&L per 1% IV Change: $0.03
   Volatility Exposure: Long Volatility
   Vega Classification: Low IV Sensitivity

THETA ANALYSIS
   Daily Time Decay: $-0.05
   Weekly Time Decay: $-0.32
   Monthly Time Decay (30d): $-1.38
   Theta Classification: Medium - Moderate time decay
   Stock Move to 

{'delta': 0.46042984962896527,
 'gamma': 0.11829112305739611,
 'vega': 0.02933749411381489,
 'theta': -0.04592883776592379,
 'rho': 0.006439306550212194,
 'buy_score': 54,
 'recommendation': 'NEUTRAL/SLIGHT BUY'}

In [23]:
# Debug: Check what's in reasonable_stocks
print("First 5 stocks:")
for i, (ticker, price) in enumerate(reasonable_stocks[0:5]):
    print(f"{i}: {ticker} - ${price}")

First 5 stocks:
0: CMCSA - $29.73
1: NU - $16.2
2: KDP - $28.59
3: SOFI - $26.58
4: KHC - $24.68


In [25]:
# Test the logic step by step
ticker = 'CMCSA'
price = 29.73

ticker_obj = yf.Ticker(ticker)
chain = ticker_obj.option_chain('2026-01-16')

# Find nearest strike for call (above current price)
call_strikes = chain.calls['strike'].values
call_target = price * 1.1
call_strike = call_strikes[np.argmin(np.abs(call_strikes - call_target))]

print(f"Price: ${price}")
print(f"Call target: ${call_target:.2f}")
print(f"Nearest call strike: ${call_strike}")

# Now try to get the option price
result = option_price(ticker, '2026-01-16', call_strike, 'call')
print(f"Call option data: {result}")

Price: $29.73
Call target: $32.70
Nearest call strike: $32.5
Call option data: {'lastPrice': 0.24, 'bid': 0.22, 'ask': 0.24, 'volume': 2218}
